In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import json
import numpy 
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import pyarrow
import sys
from tqdm import tqdm

from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
from os import path
from typing import List,Dict, Tuple
from collections import defaultdict
pd.set_option("display.max_columns", None)

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../scripts")))

import create_queries
import prepare_data

from bespoke.inventory.analysis.shared import download_util, inventory_types
from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util

%load_ext autoreload
%autoreload 2

In [3]:
today = date.today()
today

datetime.date(2022, 6, 27)

In [4]:
from underwriting import data_quality_checks,client_surveillance_jupyter

In [5]:
TRANSFER_PACKAGES_START_DATE = '2020-01-01'
SALES_TRANSACTIONS_START_DATE = '2020-01-01'

In [6]:
ca_df = pd.read_csv('df_0623.csv',index_col = 0)
ca_df.columns = ['company_name','company_identifier','state','licenses']
ca_df['company_identifier'] = ca_df['company_identifier'].astype(str)

In [7]:
ca_df_clean = ca_df[~ca_df['company_identifier'].isin(['PL','WHT','GF','UR'])]
ca_df_clean.shape

(71, 4)

In [8]:
ca_df_clean['lic_list'] = [list(ca_df_clean['licenses'])[i].split(";") for i in range(len(ca_df_clean))]

In [9]:
ca_df_clean[0:3]

,company_name,company_identifier,state,licenses,lic_list
0,99 High Tide,99HT,CA,C10-0000279-LIC,[C10-0000279-LIC]
1,Altum LLC,ALT,MI,PC-000310;AU-R-000380,"[PC-000310, AU-R-000380]"
2,Bella Sol,BS,MI,AU-R-000156;PC-000185,"[AU-R-000156, PC-000185]"


# inventory analysis

In [10]:
today = date.today()
today

datetime.date(2022, 6, 27)

In [11]:
def read_inv_data(COMPANY_IDENTIFIER,license_numbers):
    
    company_inventory_packages_query = create_queries.create_company_inventory_packages_query(
    COMPANY_IDENTIFIER,
    include_quantity_zero=True,
    license_numbers=license_numbers,
    )
    company_inventory_packages_dataframe = pd.read_sql_query(company_inventory_packages_query, engine)
    return company_inventory_packages_dataframe


In [12]:
def get_age_bucket(age):
    if age < 90:
        return '0-90'
    elif 90 <= age < 270:
        return '90-9m'
    elif 270 <= age < 365:
        return '9m-12m'
    elif 365 <= age:
        return '>12m'
    else:
        return np.nan
        

In [13]:
def get_inv_list(company_list,map_df):
    inv_df_list = {}
    for c in tqdm(company_list):
        inv_list = {}
        print(c)
        l_list = map_df[map_df['company_identifier'] == c]['lic_list'].to_list()[0]
        for l in tqdm(l_list):
            print(l)
            #df_in = read_df_in([c],[l])
            df_inv = read_inv_data([c],[l])

            if df_inv.shape[0] == 0:
                inv_list[l] = numpy.nan
                continue
            else:
                #inv = client_surveillance_jupyter.calculate_inventory_valuation(df_in,df_inv,[l],today)
                #inv_list[l] = inv
                df_inv["age"] = [today - df_inv["packaged_date"][i] for i in range(len(df_inv))]
                df_inv["age_int"] = [df_inv["age"][i] / numpy.timedelta64(1, "D") for i in range(len(df_inv))]
                df_inv = df_inv.drop(['age'], axis=1)
                df_inv['age_bucket'] = [get_age_bucket(df_inv['age_int'][i]) for i in range(len(df_inv))]
                inv_list[l] = df_inv
        inv_df_list[c] =  inv_list
    return inv_df_list

In [14]:
b = get_inv_list(list(ca_df_clean['company_identifier']),ca_df_clean)




  0%|          | 0/71 [00:00<?, ?it/s]

99HT



  0%|          | 0/1 [00:00<?, ?it/s]

C10-0000279-LIC



  1%|▏         | 1/71 [00:01<01:45,  1.51s/it]

...


 33%|███▎      | 1/3 [00:00<00:01,  1.39it/s]

C10-0000802-LIC



 67%|██████▋   | 2/3 [00:01<00:00,  1.45it/s]

C12-0000159-LIC



100%|██████████| 71/71 [02:02<00:00,  1.72s/it]


In [15]:
b.keys()

dict_keys(['99HT', 'ALT', 'BS', 'BMC', 'BBF', 'BUD', 'BRC', 'BYN', 'CHO', 'CG', 'CPA', 'CPC', 'CSC', 'CCC', 'CLC', 'CC', 'DWF', 'DCO', 'DL', 'DGG', 'DG', 'EL', 'EMA', 'EMF', 'EMM', 'EMT', 'EH', 'FW', 'GT', 'GLNR', 'GRG', 'GTR', 'GHC', 'HB', 'HS', 'HPCC', 'HC', 'IDC', 'ID', 'KAR', 'KC', 'LEG', 'LBC', 'MSS', 'ML', 'MW', 'MPW', 'MD', 'ND', 'NECC', 'PGM', 'PRP', 'PRL', 'QR', 'RA', 'RGD', 'SFV', 'SV', 'SLCC', 'SO', 'ST', 'DW', 'TGL', 'TJR', 'TTS', 'TT', 'T4L', 'TL', 'UHHC', 'VS', 'VOY'])

In [16]:
df_inv = pd.DataFrame()
for k in b.keys():
    print(k)
    for l in b[k].keys():
        print(l)
        data_inv = b[k][l]
        if str(data_inv) == 'nan':
            continue
        else:
            data_inv['company'] = k
            data_inv['location'] = l
            print(data_inv.shape)
            df_inv = df_inv.append(data_inv)
df_inv['id'] = list(zip(df_inv.company, df_inv.location))
df_inv = df_inv.reset_index(drop = True)

99HT
C10-0000279-LIC
(721, 25)
ALT
PC-000310
(104, 25)
AU-R-000380
(569, 25)
BS
AU-R-000156
(440, 25)
PC-000185
(244, 25)
BMC
402-00930
(658, 25)
BBF
MR281525
(249, 25)
BUD
C9-0000167-LIC
(4838, 25)
C9-0000444-LIC
(937, 25)
C9-0000464-LIC
(2232, 25)
C9-0000467-LIC
(1554, 25)
C9-0000399-LIC
(4229, 25)
BRC
C10-0000603-LIC
(7023, 25)
BYN
C10-0000528-LIC
(603, 25)
CHO
AU-R-000575
(1292, 25)
CG
C9-0000157-LIC
(9425, 25)
CPA
C9-0000348-LIC
(13040, 25)
C10-0000461-LIC
(3145, 25)
CPC
C9-0000056-LIC
(1294, 25)
CSC
C10-0000670-LIC
(16407, 25)
C10-0000918-LIC
(2588, 25)
CCC
C12-0000087-LIC
(23470, 25)
CLC
C10-0000986-LIC
(408, 25)
C12-0000318-LIC
(231, 25)
CC
C10-0000758-LIC
(3755, 25)
DWF
402R-00296
(729, 25)
402R-00804
(293, 25)
DCO
050-10052885D4C
(1368, 25)
050-10070593E9E
(597, 25)
DL
MR283369
(443, 25)
DGG
C9-0000016-LIC
(2105, 25)
DG
PC-000485
(4, 25)
AU-R-000287
(572, 25)
AU-R-000359
(741, 25)
AU-R-000422
(776, 25)
AU-R-000470
(422, 25)
AU-R-000572
(377, 25)
EL
C9-0000370-LIC
(313, 25)
EM

## Age distribution by package count

In [17]:
df_clients = [
'DL',
'DW',
'EMA',
'EMM',
'EMT',
'EMF',
'ST',
'GRG',
'MPW',
'EL',
'VS',
'MD',
'DWF',
'TT',
'GHC',
'SV',
'99HT',
'NECC',
'LBC',
'BYN',
'CSC',
'CHO',
'BBF',
'CCC',
'PL',
'CPA'
]

In [18]:
df_inv_df = df_inv[df_inv['company'].isin(df_clients)]

In [20]:
age_package_count = df_inv_df[['id','age_bucket','package_id']].groupby(['id','age_bucket']).count().reset_index()  

In [21]:
age_package_count['perc'] = age_package_count[['id','package_id']].groupby(['id']).apply(lambda x:100 * x / float(x.sum()))

In [22]:
age_package_count.columns = ['id', 'age_bucket', 'count_pkg', 'perc']

In [23]:
age_package_count.head()

,id,age_bucket,count_pkg,perc
0,"(99HT, C10-0000279-LIC)",0-90,406,56.310680
1,"(99HT, C10-0000279-LIC)",90-9m,144,19.972261
2,"(99HT, C10-0000279-LIC)",9m-12m,65,9.015257
3,"(99HT, C10-0000279-LIC)",>12m,106,14.701803
4,"(BBF, MR281525)",0-90,114,45.783133


In [24]:
age_90 = age_package_count[age_package_count['age_bucket'] == '0-90']
age_909m = age_package_count[age_package_count['age_bucket'] == '90-9m']
age_9m12m = age_package_count[age_package_count['age_bucket'] == '9m-12m']
age_12m = age_package_count[age_package_count['age_bucket'] == '>12m']

In [25]:
age_ = age_90.merge(age_909m,how = 'outer',on = 'id').\
merge(age_9m12m,how = 'outer',on = 'id').\
merge(age_12m,how = 'outer',on = 'id')

In [26]:
age_.columns = ['id', 'age_bucket_90', 'count_pkg_90', 'perc_90', 
                     'age_bucket_909m','count_pkg_909m', 'perc_909m',
                    'age_bucket_9m12m','count_pkg_9m12m', 'perc_9m12m',
                     'age_bucket_12m','count_pkg_12m', 'perc_12m'
                    ]

In [29]:
age_['new_fresh'] = age_['perc_90'] + age_['perc_909m']

In [30]:
age_.head()

,id,age_bucket_90,count_pkg_90,perc_90,age_bucket_909m,count_pkg_909m,perc_909m,age_bucket_9m12m,count_pkg_9m12m,perc_9m12m,age_bucket_12m,count_pkg_12m,perc_12m,new_fresh
0,"(99HT, C10-0000279-LIC)",0-90,406,56.310680,90-9m,144,19.972261,9m-12m,65.0,9.015257,>12m,106.0,14.701803,76.282940
1,"(BBF, MR281525)",0-90,114,45.783133,90-9m,113,45.381526,9m-12m,14.0,5.622490,>12m,8.0,3.212851,91.164659
2,"(BYN, C10-0000528-LIC)",0-90,321,53.233831,90-9m,242,40.132670,9m-12m,27.0,4.477612,>12m,13.0,2.155887,93.366501
3,"(CCC, C12-0000087-LIC)",0-90,2107,8.977418,90-9m,4054,17.273115,9m-12m,1663.0,7.085641,>12m,15646.0,66.663826,26.250533
4,"(CHO, AU-R-000575)",0-90,361,27.941176,90-9m,913,70.665635,9m-12m,16.0,1.238390,>12m,2.0,0.154799,98.606811


In [83]:
age_.to_csv('df_inv_analysis_pkg.csv')

## Inventory valuation

In [31]:
def read_df_in(COMPANY_IDENTIFIER,license_numbers):
    company_incoming_transfer_packages_query = create_queries.create_company_incoming_transfer_packages_query(
    COMPANY_IDENTIFIER,
    TRANSFER_PACKAGES_START_DATE,
    license_numbers=license_numbers,
    )
    company_incoming_transfer_packages_dataframe = pd.read_sql_query(company_incoming_transfer_packages_query, engine)
    return company_incoming_transfer_packages_dataframe


In [1]:
def calculate_inventory_valuation_fresh(
    incoming_transfer_df, inventory_df, license_list, today_date,days_low,days_high
):
    # legal name
    legal_name = incoming_transfer_df[
        incoming_transfer_df["license_number"].isin(license_list)
    ]["recipient_facility_name"].values[0]
    # process df_in and df_sales
    incoming_transfer_df["per_unit_incoming"] = (
        incoming_transfer_df["shipper_wholesale_price"]
        / incoming_transfer_df["shipped_quantity"]
    )
    incoming_transfer_df_price = incoming_transfer_df[
        incoming_transfer_df["shipper_wholesale_price"].notnull()
    ]
    # by package id
    average_incoming_package_id = incoming_transfer_df_price.groupby(["package_id"])[
        "per_unit_incoming"
    ].mean()
    df_avg_incoming_price = pd.Series(average_incoming_package_id).to_frame()
    df_avg_incoming_price = df_avg_incoming_price.reset_index()
    # by product
    average_incoming_product = incoming_transfer_df_price.groupby(["product_name"])[
        "per_unit_incoming"
    ].mean()
    df_avg_product = pd.Series(average_incoming_product).to_frame()
    df_avg_product = df_avg_product.reset_index()
    df_avg_product.rename(
        columns={"per_unit_incoming": "per_unit_product"}, inplace=True
    )
    inventory_df = get_valid_inventory_df(inventory_df)

    # prepare fresh inventory
    inventory_df = inventory_df.reset_index(drop=True)
    inventory_df["age"] = [
        today_date - inventory_df["packaged_date"][i] for i in range(len(inventory_df))
    ]
    inventory_df["age_int"] = [
        inventory_df["age"][i] / numpy.timedelta64(1, "D")
        for i in range(len(inventory_df))
    ]
    inventory_df_fresh = inventory_df[(inventory_df["age_int"] >= days_low)&(inventory_df["age_int"] < days_high)]

    # calculate inventory
    df_inventory_incoming = pd.merge(
        inventory_df_fresh,
        df_avg_incoming_price,
        left_on=["package_id"],
        right_on=["package_id"],
        how="left",
    )
    # left_on=['tx_product_name','tx_unit_of_measure'], right_on=['product_name','shipped_unit_of_measure'], how='left'
    df_inventory_incoming.replace([numpy.inf], numpy.nan, inplace=True)
    df_inv_null = df_inventory_incoming[
        df_inventory_incoming["per_unit_incoming"].isnull()
    ]
    df_inv_product = pd.merge(
        df_inv_null,
        df_avg_product,
        left_on=["product_name"],
        right_on=["product_name"],
        how="left",
    )
    df_inv_product.replace([numpy.inf], numpy.nan, inplace=True)
    df_inv_product_price = df_inv_product[df_inv_product["per_unit_product"].notnull()]
    df_inv_product_price["total_price"] = (
        df_inv_product_price["quantity"] * df_inv_product_price["per_unit_product"]
    )

    inventory_product_value = df_inv_product_price["total_price"].sum()
    df_inventory_incoming["total_price"] = (
        df_inventory_incoming["quantity"] * df_inventory_incoming["per_unit_incoming"]
    )
    inventory_value = df_inventory_incoming["total_price"].sum()
    total_inv_value = inventory_product_value + inventory_value
    total_inv_value_after_tax = (inventory_product_value + inventory_value) * 1.27
    inv_count_product = df_inv_product_price["per_unit_product"].count()
    inv_count_incoming = df_inventory_incoming["per_unit_incoming"].count()
    inv_count_total = df_inventory_incoming["quantity"].count()
    inv_total_incoming = inv_count_product + inv_count_incoming
    inventory_coverage = inv_total_incoming / inv_count_total
    # prepare data
    data = [
        [today_date],
        [round(total_inv_value, 2)],
        [round(total_inv_value_after_tax, 2)],
        [inv_total_incoming],
        [inv_count_total],
        [round(inventory_coverage, 2)],
        [license_list],
        [legal_name],
    ]
    df_inventory_license = pd.DataFrame(data).T
    df_inventory_license.columns = [
        "date",
        "value"+str(days_high),
        "value_after_tax",
        "total_incoming",
        "total",
        "coverage",
        "license",
        "legal_name",
    ]
    return df_inventory_license, inventory_df

In [62]:
def get_inv_val_list(company_list,map_df):
    inv_df_list_90 = {}
    inv_df_list_9m = {}
    inv_df_list_12m = {}
    inv_df_list_12m_plus = {}
    for c in tqdm(company_list):
        inv_list_90 = {}
        inv_list_9m = {}
        inv_list_12m = {}
        inv_list_12m_plus = {}
        print(c)
        l_list = map_df[map_df['company_identifier'] == c]['lic_list'].to_list()[0]
        for l in tqdm(l_list):
            print(l)
            df_in = read_df_in([c],[l])
            df_inv = read_inv_data([c],[l])
            if (df_inv.shape[0] == 0) or df_in.shape[0] == 0:
                inv_list_90[l] = numpy.nan
                inv_list_9m[l] = numpy.nan
                inv_list_12m[l] = numpy.nan
                inv_list_12m_plus[l] = numpy.nan
                continue
            else:
                inv_90 = client_surveillance_jupyter.calculate_inventory_valuation_fresh(df_in,df_inv,[l],today,0,90)[0]
                inv_9m = client_surveillance_jupyter.calculate_inventory_valuation_fresh(df_in,df_inv,[l],today,90,270)[0]
                inv_12m = client_surveillance_jupyter.calculate_inventory_valuation_fresh(df_in,df_inv,[l],today,270,365)[0]
                inv_12m_plus = client_surveillance_jupyter.calculate_inventory_valuation_fresh(df_in,df_inv,[l],today,365,10000)[0]
                inv_list_90[l] = inv_90
                inv_list_9m[l] = inv_9m
                inv_list_12m[l] = inv_12m
                inv_list_12m_plus[l] = inv_12m_plus
        inv_df_list_90[c] =  inv_list_90
        inv_df_list_9m[c] =  inv_list_9m
        inv_df_list_12m[c] =  inv_list_12m
        inv_df_list_12m_plus[c] =  inv_list_12m_plus
    return inv_df_list_90,inv_df_list_9m,inv_df_list_12m,inv_df_list_12m_plus




In [63]:
t1,t2,t3,t4 = get_inv_val_list(list(ca_df_clean['company_identifier']),ca_df_clean)




  0%|          | 0/71 [00:00<?, ?it/s]

99HT



  0%|          | 0/1 [00:00<?, ?it/s]

C10-0000279-LIC



  1%|▏         | 1/71 [00:01<02:09,  1.85s/it]

...


 33%|███▎      | 1/3 [00:04<00:09,  4.84s/it]

C10-0000802-LIC



 67%|██████▋   | 2/3 [00:08<00:04,  4.28s/it]

C12-0000159-LIC



100%|██████████| 71/71 [09:39<00:00,  8.17s/it]


In [69]:
t4.keys()

dict_keys(['99HT', 'ALT', 'BS', 'BMC', 'BBF', 'BUD', 'BRC', 'BYN', 'CHO', 'CG', 'CPA', 'CPC', 'CSC', 'CCC', 'CLC', 'CC', 'DWF', 'DCO', 'DL', 'DGG', 'DG', 'EL', 'EMA', 'EMF', 'EMM', 'EMT', 'EH', 'FW', 'GT', 'GLNR', 'GRG', 'GTR', 'GHC', 'HB', 'HS', 'HPCC', 'HC', 'IDC', 'ID', 'KAR', 'KC', 'LEG', 'LBC', 'MSS', 'ML', 'MW', 'MPW', 'MD', 'ND', 'NECC', 'PGM', 'PRP', 'PRL', 'QR', 'RA', 'RGD', 'SFV', 'SV', 'SLCC', 'SO', 'ST', 'DW', 'TGL', 'TJR', 'TTS', 'TT', 'T4L', 'TL', 'UHHC', 'VS', 'VOY'])

In [64]:
def decompress_inv_dict(inv_dict):
    df_decompress = pd.DataFrame()
    for k in inv_dict.keys():
        print(k)
        for l in inv_dict[k].keys():
            print(l)
            data_inv_val = inv_dict[k][l]
            if str(data_inv_val) == 'nan':
                continue
            else:
                data_inv_val['company'] = k
                data_inv_val['location'] = l
                print(data_inv_val.shape)
                df_decompress = df_decompress.append(data_inv_val)
    return df_decompress
    

In [65]:
df_decompress_90 = decompress_inv_dict(t1)
df_decompress_9m = decompress_inv_dict(t2)
df_decompress_12m = decompress_inv_dict(t3)
df_decompress_12m_plus = decompress_inv_dict(t4)

99HT
C10-0000279-LIC
(1, 10)
ALT
PC-000310
(1, 10)
AU-R-000380
(1, 10)
BS
AU-R-000156
(1, 10)
PC-000185
(1, 10)
BMC
402-00930
(1, 10)
BBF
MR281525
(1, 10)
BUD
C9-0000167-LIC
(1, 10)
C9-0000444-LIC
(1, 10)
C9-0000464-LIC
(1, 10)
C9-0000467-LIC
(1, 10)
C9-0000399-LIC
(1, 10)
BRC
C10-0000603-LIC
(1, 10)
BYN
C10-0000528-LIC
(1, 10)
CHO
AU-R-000575
(1, 10)
CG
C9-0000157-LIC
(1, 10)
CPA
C9-0000348-LIC
(1, 10)
C10-0000461-LIC
(1, 10)
CPC
C9-0000056-LIC
(1, 10)
CSC
C10-0000670-LIC
(1, 10)
C10-0000918-LIC
(1, 10)
CCC
C12-0000087-LIC
(1, 10)
CLC
C10-0000986-LIC
C12-0000318-LIC
(1, 10)
CC
C10-0000758-LIC
(1, 10)
DWF
402R-00296
(1, 10)
402R-00804
(1, 10)
DCO
050-10052885D4C
(1, 10)
050-10070593E9E
(1, 10)
DL
MR283369
(1, 10)
DGG
C9-0000016-LIC
(1, 10)
DG
PC-000485
(1, 10)
AU-R-000287
(1, 10)
AU-R-000359
(1, 10)
AU-R-000422
(1, 10)
AU-R-000470
(1, 10)
AU-R-000572
(1, 10)
EL
C9-0000370-LIC
(1, 10)
EMA
C10-0000774-LIC
(1, 10)
EMF
C10-0000939-LIC
(1, 10)
EMM
C10-0000786-LIC
(1, 10)
EMT
C10-0000695-LIC

(1, 10)
DL
MR283369
(1, 10)
DGG
C9-0000016-LIC
(1, 10)
DG
PC-000485
(1, 10)
AU-R-000287
(1, 10)
AU-R-000359
(1, 10)
AU-R-000422
(1, 10)
AU-R-000470
(1, 10)
AU-R-000572
(1, 10)
EL
C9-0000370-LIC
(1, 10)
EMA
C10-0000774-LIC
(1, 10)
EMF
C10-0000939-LIC
(1, 10)
EMM
C10-0000786-LIC
(1, 10)
EMT
C10-0000695-LIC
(1, 10)
EH
MR283073
(1, 10)
FW
402-00558
(1, 10)
402-00891
(1, 10)
402R-00017
(1, 10)
402R-00020
(1, 10)
402R-00747
(1, 10)
GT
C9-0000170-LIC
(1, 10)
C9-0000385-LIC
(1, 10)
GLNR
PC-000334
(1, 10)
PC-000613
(1, 10)
PC-000722
(1, 10)
AU-R-000182
(1, 10)
AU-R-000461
(1, 10)
AU-R-000559
(1, 10)
GRG
C10-0000169-LIC
(1, 10)
C10-0000408-LIC
(1, 10)
GTR
PC-000225
(1, 10)
AU-R-000163
(1, 10)
GHC
C10-0000414-LIC
(1, 10)
HB
C10-0000339-LIC
(1, 10)
C10-0000624-LIC
(1, 10)
C10-0000301-LIC
(1, 10)
HS
C9-0000441-LIC
(1, 10)
HPCC
C10-0000064-LIC
(1, 10)
C10-0000005-LIC
(1, 10)
HC
C9-0000341-LIC
(1, 10)
IDC
C9-0000320-LIC
(1, 10)
ID
C9-0000166-LIC
(1, 10)
KAR
402-00693
(1, 10)
402R-00744
(1, 10)
KC
402

In [75]:
df_decompress_12m_plus.head()

,date,value10000,value_after_tax,total_incoming,total,coverage,license,legal_name,company,location
0,2022-06-27,2508.87,3186.26,86,103,0.83,[C10-0000279-LIC],99 HIGHTIDE COLLECTIVE,99HT,C10-0000279-LIC
0,2022-06-27,2615.0,3321.05,6,6,1.0,[PC-000310],Pure Canna Michigan,ALT,PC-000310
0,2022-06-27,1900.96,2414.22,8,8,1.0,[AU-R-000380],PURE CANNA MICHIGAN,ALT,AU-R-000380
0,2022-06-27,1022.0,1297.94,9,10,0.9,[AU-R-000156],"Bella Sol Wellness Centers of Michigan, LLC",BS,AU-R-000156
0,2022-06-27,4418.61,5611.63,31,35,0.89,[PC-000185],"Bella Sol Wellness Centers of Michigan, LLC",BS,PC-000185


In [80]:
age_val = df_decompress_90[['company','location','value90']].\
merge(df_decompress_9m[['company','location','value270']],on = ['company','location'],how = 'outer').\
merge(df_decompress_12m[['company','location','value365']],on = ['company','location'],how = 'outer').\
merge(df_decompress_12m_plus[['company','location','value10000']],on = ['company','location'],how = 'outer')

age_val['id'] = list(zip(age_val.company, age_val.location))
age_val = age_val.reset_index(drop = True)



In [81]:
age_val

,company,location,value90,value270,value365,value10000,id
0,99HT,C10-0000279-LIC,90529.27,13935.79,3660.34,2508.87,"(99HT, C10-0000279-LIC)"
1,ALT,PC-000310,19954.0,13821.36,1177.84,2615.0,"(ALT, PC-000310)"
2,ALT,AU-R-000380,253047.85,61858.52,2627.03,1900.96,"(ALT, AU-R-000380)"
3,BS,AU-R-000156,17859.75,14917.21,714.65,1022.0,"(BS, AU-R-000156)"
4,BS,PC-000185,362.66,16039.02,400.0,4418.61,"(BS, PC-000185)"
...,...,...,...,...,...,...,...
142,VS,402-00840,28046.2,7827.06,260.74,0.0,"(VS, 402-00840)"
143,VS,402R-00545,124816.99,6695.11,48.0,0.0,"(VS, 402R-00545)"
144,VOY,C12-0000030-LIC,8155.54,312.57,30.96,0.2,"(VOY, C12-0000030-LIC)"
145,VOY,C10-0000802-LIC,46532.74,7943.36,183.05,0.0,"(VOY, C10-0000802-LIC)"


In [84]:
age_val_df = age_val[age_val['company'].isin(df_clients)]

In [85]:
age_val_df.to_csv('df_inv_analysis_val.csv')

## Consider other bucket cutoffs?

In [99]:
age_distr = df_inv_df[['company','location','age_int']].groupby(['company','location']).describe().reset_index()

In [102]:
#age_distr.to_csv('df_inv_age_distr.csv')

## age by product category

In [111]:
df_inv_df['company'].unique()

array(['99HT', 'BBF', 'BYN', 'CHO', 'CPA', 'CSC', 'CCC', 'DWF', 'DL',
       'EL', 'EMA', 'EMF', 'EMM', 'EMT', 'GRG', 'GHC', 'LBC', 'MPW', 'MD',
       'NECC', 'SV', 'ST', 'DW', 'TT', 'VS'], dtype=object)

In [104]:
df_inv_df_with_state = pd.merge(df_inv_df,ca_df_clean[['company_identifier','state']],left_on = 'company',right_on = 'company_identifier',how = 'left')



In [112]:
age_distr_by_product = df_inv_df_with_state[['product_category_name','age_int']].groupby(['product_category_name']).describe()

In [109]:
age_distr_by_product.to_csv('age_distr_by_product.csv')

In [113]:
age_distr_by_product

age_int                                 \
                                         count        mean         std    min   
product_category_name                                                           
Buds                                     980.0   98.271429   84.089863    3.0   
Buds (prepackaged)                        96.0  144.864583   59.824061   40.0   
Capsule (weight - each)                  585.0  355.758974  252.041279    7.0   
Clone - Cutting                          311.0  407.684887  247.242294    3.0   
Clone - Tissue Culture                    18.0  406.222222  145.299106   20.0   
Concentrate                              568.0   83.267606  103.317470    4.0   
Concentrate (Bulk)                         4.0  109.000000    0.000000  109.0   
Concentrate (Each)                       804.0   88.521144   97.868521    4.0   
Edible (volume - each)                  2994.0  298.231129  218.179951    3.0   
Edible (weight - each)                 17308.0  326.273631  247.754279    3.0   
Extract (volume - each)                   68.0  310.823529  198.691010   14.0   
Extract (weight - each)                 8386.0  357.465657  243.321338    4.0   
Extract (weight)                          18.0  213.444444  219.527627   70.0   
Flower                                  2128.0  765.540883  138.246821   20.0   
Flower (packaged - each)                 773.0  192.068564  152.605647    3.0   
Flower (packaged eighth - each)        16459.0  267.745063  198.056262    3.0   
Flower (packaged gram - each)           1352.0  283.892012  207.406170    3.0   
Flower (packaged half ounce - each)     2075.0  271.039036  194.564063    4.0   
Flower (packaged ounce - each)          1060.0  214.067925  168.975358    3.0   
Flower (packaged quarter - each)        1319.0  274.699014  203.999047    4.0   
Immature Plant                            76.0  705.842105  240.993060   82.0   
Immature Plants                            4.0    8.000000    6.164414    3.0   
Infused (edible)                        1104.0   95.182971  134.844530    5.0   
Infused (non-edible)                      83.0  128.939759  128.290576    3.0   
Infused Beverage                          64.0   72.609375   86.521420   12.0   
Infused Butter/Oil (weight - each)        30.0  214.200000  152.643917   11.0   
Infused Liquid (each)                     17.0  183.647059   70.170276   84.0   
Infused Non-Edible Liquid                  2.0  137.500000   48.790368  103.0   
Infused Non-Edible Solid                  29.0  115.137931   30.460424   68.0   
Infused Pre-Rolls                         40.0  107.275000   31.782907   27.0   
Infused-Edible                           305.0  145.377049   77.441609   12.0   
Inhalable Compound Concentrate             4.0  167.750000   90.680299   34.0   
Inhalable Compound Concentrate (each)     46.0  100.978261   60.392232   12.0   
Kief                                      11.0  193.090909  226.609115   34.0   
Other Concentrate (volume - each)        262.0  236.293893  199.682069    5.0   
Other Concentrate (weight - each)       4903.0  282.776871  231.714029    3.0   
Other Concentrate (weight)                 4.0  390.500000  280.723470  192.0   
Pre-Roll Flower                         9393.0  338.190780  249.052949    3.0   
Pre-Roll Infused                        8197.0  232.550689  185.378787    4.0   
Pre-Roll Leaf                            911.0  356.801317  207.755946   13.0   
Raw Pre-Rolls                            365.0   77.241096   57.065385    3.0   
Seeds                                     11.0   65.363636   32.794124   24.0   
Seeds (each)                             294.0  319.622449  186.616461   18.0   
Shake                                     28.0  251.857143  231.259949   40.0   
Shake (Packaged Eighth - each)            20.0  318.750000  150.983399   82.0   
Shake (Packaged Gram - each)              57.0  167.473684  195.061996   11.0   
Shake (Packaged Half Ounce - each)       128.0  271.671875  170.873030